In [ ]:
import rtrtme
import pandas as pd
import json
import numpy as np

In [ ]:
data_dir = "data"

In [ ]:
client = rtrtme.RTRTMePy()

In [ ]:
client.load_config()

In [ ]:
event_name = "IRM-OMAN703-2025"

In [ ]:
event = client.get(f"events/{event_name}")

In [ ]:
event_date = event["date"].replace("-", "")

In [ ]:
json.dump(event, open(f"{data_dir}/json/{event_date}_{event_name.lower()}_event.json", "wt"), indent=4, ensure_ascii=False)

In [ ]:
pages = client.get_list(f"events/{event_name}/profiles", max=2000)

In [ ]:
json.dump(pages, open(f"{data_dir}/json/{event_date}_{event_name.lower()}_profiles.json", "wt"), indent=4, ensure_ascii=False)

In [ ]:
profiles = [profile for page in pages for profile in page["list"]]

In [ ]:
results = list()
for i in range(0, len(profiles)):
    print(f"Fetching {i + 1} of {len(profiles)}...")
    try:
        data = client._get(f"events/{event_name}/profiles/{profiles[i]['pid']}/splits", quite=True)
        results.append(data)
    except rtrtme.APIException:
        print(f"No results for {profiles[i]['pid']} with class {profiles[i]['class']}?")

In [ ]:
json.dump(results, open(f"{data_dir}/json/{event_date}_{event_name.lower()}_results.json", "wt"), indent=4)

In [ ]:
# Can reload to adjust transformation rather than re-request everything
pages = json.load(open(f"{data_dir}/json/{event_date}_{event_name.lower()}_results.json", "rt"))

In [ ]:
df = pd.DataFrame([split for page in pages for result in page for split in result["list"]])

In [ ]:
df["splitTimeSeconds"] = pd.to_timedelta(df["splitTime"])
df["legTimeSeconds"] = pd.to_timedelta(df["legTime"])
df["netTimeSeconds"] = pd.to_timedelta(df["netTime"])
df["waveTimeSeconds"] = pd.to_timedelta(df["waveTime"])

df["splitTimeSeconds"] = df["splitTimeSeconds"].dt.total_seconds()
df["legTimeSeconds"] = df["legTimeSeconds"].dt.total_seconds()
df["netTimeSeconds"] = df["netTimeSeconds"].dt.total_seconds()
df["waveTimeSeconds"] = df["waveTimeSeconds"].dt.total_seconds()

df.loc[df["point"] == "START", "splitTimeSeconds"] = df["splitTimeSeconds"].fillna(0)
df.loc[df["point"] == "START", "legTimeSeconds"] = df["legTimeSeconds"].fillna(0)
df.loc[df["point"] == "START", "netTimeSeconds"] = df["netTimeSeconds"].fillna(0)
df.loc[df["point"] == "START", "waveTimeSeconds"] = df["waveTimeSeconds"].fillna(0)

In [ ]:
df = df.drop(columns=["bib_display", "tag", "startTime", "profile_pic", "profile_color", "_ver", "alias", "course", "i", "u", "etnp", "etfp", "results"])

In [ ]:
df.loc[df["division"].str.upper().str.contains("RELAY"), "divisionType"] = "RELAY"
df.loc[df["division"].str.upper().str.contains("ODIV"), "divisionType"] = "OPEN"
df.loc[df["division"].str.upper().str.contains("PC"), "divisionType"] = "PC"
df.loc[df["division"].str.upper().str.contains("GUIDE"), "divisionType"] = "GUIDE"
df.loc[df["division"].str.upper().str.contains("PRO"), "divisionType"] = "PRO"
df["divisionType"] = df["divisionType"].fillna("AG")
#df["divisionType"] = df["divisionType"].fillna("PRO-AG")

In [ ]:
df["legDivPos"] = df.loc[df["point"].isin(["SWIM", "T1", "BIKE", "T2", "FINISH"]), :].groupby(by=["point", "divisionType", "division"])["legTimeSeconds"].rank("dense")
df["legSexPos"] = df.loc[df["point"].isin(["SWIM", "T1", "BIKE", "T2", "FINISH"]), :].groupby(by=["point", "divisionType", "sex"])["legTimeSeconds"].rank("dense")
df["legOvrPos"] = df.loc[df["point"].isin(["SWIM", "T1", "BIKE", "T2", "FINISH"]), :].groupby(by=["point", "divisionType"])["legTimeSeconds"].rank("dense")

df["finDivPos"] = df.loc[df["point"] == "FINISH", :].groupby(by=["point", "divisionType", "division"])["netTimeSeconds"].rank("dense")
df["finSexPos"] = df.loc[df["point"] == "FINISH", :].groupby(by=["point", "divisionType", "sex"])["netTimeSeconds"].rank("dense")
df["finOvrPos"] = df.loc[df["point"] == "FINISH", :].groupby(by=["point", "divisionType"])["netTimeSeconds"].rank("dense")

In [ ]:
points = df.groupby("point")[["netTimeSeconds"]].min()
points["pointNum"] = points["netTimeSeconds"].rank(method="dense").astype("int64")
points = points.drop(columns=["netTimeSeconds"])

In [ ]:
df = df.join(points["pointNum"], on="point")

In [ ]:
df["maxPointNum"] = df.groupby(by=["pid"])["pointNum"].transform("max")

In [ ]:
df["maxPointNetTimeSeconds"] = df.loc[df["pointNum"] == df["maxPointNum"], "netTimeSeconds"]
df["maxPointNetTimeSeconds"] = df.groupby(by=["pid"])["maxPointNetTimeSeconds"].transform("min")

In [ ]:
df = df.sort_values(by=["maxPointNum", "maxPointNetTimeSeconds"], ascending=[False, True])
df["rank"] = np.arange(len(df)) + 1
df["rank"] = df.groupby(by=["maxPointNum", "maxPointNetTimeSeconds"])["rank"].transform("mean")
df["rank"] = df["rank"].rank(method="dense")

In [ ]:
df = df.sort_values(by=["rank", "pointNum"])

In [ ]:
finishPointNum = df["pointNum"].max()
df["isDNF"] = df["maxPointNum"].values != finishPointNum

In [ ]:
df.reset_index(inplace=True, drop=True)

In [ ]:
df.to_csv(f"{data_dir}/csv/{event_date}_{event_name.lower()}_tall.csv")

In [ ]:
df.loc[df["point"] == "SWIMEND", "point"] = "SWIM"
df.loc[df["point"] == "BIKESTART", "point"] = "T1"
df.loc[df["point"] == "BIKEEND", "point"] = "BIKE"
df.loc[df["point"] == "RUNSTART", "point"] = "T2"

In [ ]:
df = df.loc[df["point"].isin(["SWIM", "T1", "BIKE", "T2", "FINISH"]), :]
df["point"] = df["point"].str.lower()
df.loc[:, "finDivPos"] = df.groupby(by=["pid"])["finDivPos"].transform("max")
df.loc[:, "finSexPos"] = df.groupby(by=["pid"])["finSexPos"].transform("max")
df.loc[:, "finOvrPos"] = df.groupby(by=["pid"])["finOvrPos"].transform("max")

In [ ]:
df.rename(columns={"legTime": "LegTime", "netTime": "NetTime"}, inplace=True)

In [ ]:
# Chunk by athlete to do the pivot
dfs = [y for x, y in df.groupby('pid')]

In [ ]:
for i in range(0, len(dfs)):
    dfs[i] = pd.pivot_table(
        dfs[i],
        values=["LegTime", "NetTime"],
        index=["rank", "bib", "name", "country", "sex", "division", "divisionType", "finDivPos", "finSexPos", "finOvrPos", "isDNF"],
        columns="point",
        aggfunc="max",
        dropna=False,
    )

In [ ]:
df = pd.concat(dfs)

In [ ]:
df = df.swaplevel(0, 1, axis=1).sort_index(axis=1)

In [ ]:
df = df.reindex(level="point", columns=["swim", "t1", "bike", "t2", "finish"])

In [ ]:
df.reset_index(inplace=True)

In [ ]:
# From: https://stackoverflow.com/a/66829689
df = df.pipe(lambda s: s.set_axis(s.columns.map("".join), axis=1))

In [ ]:
df = df.sort_values(by=["rank"])

In [ ]:
df.reset_index(inplace=True, drop=True)

In [ ]:
df.to_csv(f"{data_dir}/csv/{event_date}_{event_name.lower()}_wide.csv")